In [1]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 데이터 불러오기

## 효준이 감성 점수 포함된 데이터 불러오기

In [32]:
df = pd.read_csv('분석용데이터_utf_spi수정완료.csv')

In [33]:
df2 = pd.read_csv('df_verb.csv')

In [34]:
df2

,Unnamed: 0,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,next_spi,sieon_sent_score
0,0,1676.4,4.52,2.75,3.4,2069.73,1119.25,82.5,-1.635618e+08
1,1,1674.4,4.72,2.75,3.9,1939.30,1124.65,83.4,-1.777621e+08
2,2,1677.5,4.54,3.00,4.1,2106.70,1095.50,83.9,-1.980306e+08
3,3,1684.8,4.54,3.00,3.8,2192.36,1068.00,84.1,-1.898849e+08
4,4,1690.5,4.44,3.00,3.9,2142.47,1078.00,84.4,-2.402702e+08
...,...,...,...,...,...,...,...,...,...
115,115,3100.4,2.19,0.50,0.7,2326.17,1187.94,114.0,3.239718e+07
116,116,3115.2,2.24,0.50,1.0,2327.89,1164.65,115.6,4.511543e+07
117,117,3152.8,2.24,0.50,0.1,2267.15,1136.49,118.3,4.169356e+07
118,118,3183.5,2.25,0.50,0.6,2591.34,1109.32,121.4,4.231788e+07


In [35]:
b = df2[['sieon_sent_score']]

In [48]:
b.index = df.index

In [49]:
b

,sieon_sent_score
Date,
201101월,-1.635618e+08
201102월,-1.777621e+08
201103월,-1.980306e+08
201104월,-1.898849e+08
201105월,-2.402702e+08
...,...
202008월,3.239718e+07
202009월,4.511543e+07
202010월,4.169356e+07


In [36]:
#Date를 index로 넣어줌
df.index = df['Date']
df.drop(['Date'], axis = 1, inplace = True)

In [50]:
df

,sent_score,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,효준감성점수,spi
Date,,,,,,,,,
201101월,-6.268386e+07,1676.4,4.52,2.75,3.4,2069.73,1119.25,1856.746125,81.3
201102월,-6.255832e+07,1674.4,4.72,2.75,3.9,1939.30,1124.65,2046.702906,82.5
201103월,-6.957441e+07,1677.5,4.54,3.00,4.1,2106.70,1095.50,1715.795289,83.4
201104월,-7.305733e+07,1684.8,4.54,3.00,3.8,2192.36,1068.00,1635.262791,83.9
201105월,-9.356633e+07,1690.5,4.44,3.00,3.9,2142.47,1078.00,1922.518709,84.1
...,...,...,...,...,...,...,...,...,...
202008월,-5.824726e+07,3100.4,2.19,0.50,0.7,2326.17,1187.94,1221.939746,113.1
202009월,-6.441351e+07,3115.2,2.24,0.50,1.0,2327.89,1164.65,1208.597571,114.0
202010월,-3.751069e+07,3152.8,2.24,0.50,0.1,2267.15,1136.49,1001.099665,115.6


## 시언이 감성 점수 불러오기

In [38]:
temp = pd.read_csv('news_with_sent_score.csv')
temp.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [39]:
sent_score = temp['sent_score']

In [40]:
sent_score

0     -6.268386e+07
1     -6.255832e+07
2     -6.957441e+07
3     -7.305733e+07
4     -9.356633e+07
           ...     
115   -5.824726e+07
116   -6.441351e+07
117   -3.751069e+07
118   -3.147741e+07
119   -3.986893e+07
Name: sent_score, Length: 120, dtype: float64

In [41]:
del temp

## 효준이 감성 점수 포함 데이터 프레임에 시언이 감성 점수 포함시키기

In [42]:
sent_score.index = df.index

In [43]:
df = pd.concat([sent_score, df], axis = 1)

In [44]:
df

,sent_score,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,효준감성점수,spi
Date,,,,,,,,,
201101월,-6.268386e+07,1676.4,4.52,2.75,3.4,2069.73,1119.25,1856.746125,81.3
201102월,-6.255832e+07,1674.4,4.72,2.75,3.9,1939.30,1124.65,2046.702906,82.5
201103월,-6.957441e+07,1677.5,4.54,3.00,4.1,2106.70,1095.50,1715.795289,83.4
201104월,-7.305733e+07,1684.8,4.54,3.00,3.8,2192.36,1068.00,1635.262791,83.9
201105월,-9.356633e+07,1690.5,4.44,3.00,3.9,2142.47,1078.00,1922.518709,84.1
...,...,...,...,...,...,...,...,...,...
202008월,-5.824726e+07,3100.4,2.19,0.50,0.7,2326.17,1187.94,1221.939746,113.1
202009월,-6.441351e+07,3115.2,2.24,0.50,1.0,2327.89,1164.65,1208.597571,114.0
202010월,-3.751069e+07,3152.8,2.24,0.50,0.1,2267.15,1136.49,1001.099665,115.6


In [52]:
final_df = pd.concat([df,b], axis = 1)

In [54]:
final_df

,sent_score,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,효준감성점수,spi,sieon_sent_score
Date,,,,,,,,,,
201101월,-6.268386e+07,1676.4,4.52,2.75,3.4,2069.73,1119.25,1856.746125,81.3,-1.635618e+08
201102월,-6.255832e+07,1674.4,4.72,2.75,3.9,1939.30,1124.65,2046.702906,82.5,-1.777621e+08
201103월,-6.957441e+07,1677.5,4.54,3.00,4.1,2106.70,1095.50,1715.795289,83.4,-1.980306e+08
201104월,-7.305733e+07,1684.8,4.54,3.00,3.8,2192.36,1068.00,1635.262791,83.9,-1.898849e+08
201105월,-9.356633e+07,1690.5,4.44,3.00,3.9,2142.47,1078.00,1922.518709,84.1,-2.402702e+08
...,...,...,...,...,...,...,...,...,...,...
202008월,-5.824726e+07,3100.4,2.19,0.50,0.7,2326.17,1187.94,1221.939746,113.1,3.239718e+07
202009월,-6.441351e+07,3115.2,2.24,0.50,1.0,2327.89,1164.65,1208.597571,114.0,4.511543e+07
202010월,-3.751069e+07,3152.8,2.24,0.50,0.1,2267.15,1136.49,1001.099665,115.6,4.169356e+07


In [55]:
final_df.to_csv('df_0507.csv')

In [56]:
a = pd.read_csv('df_0507.csv')

In [57]:
a

,Date,sent_score,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,효준감성점수,spi,sieon_sent_score
0,201101월,-6.268386e+07,1676.4,4.52,2.75,3.4,2069.73,1119.25,1856.746125,81.3,-1.635618e+08
1,201102월,-6.255832e+07,1674.4,4.72,2.75,3.9,1939.30,1124.65,2046.702906,82.5,-1.777621e+08
2,201103월,-6.957441e+07,1677.5,4.54,3.00,4.1,2106.70,1095.50,1715.795289,83.4,-1.980306e+08
3,201104월,-7.305733e+07,1684.8,4.54,3.00,3.8,2192.36,1068.00,1635.262791,83.9,-1.898849e+08
4,201105월,-9.356633e+07,1690.5,4.44,3.00,3.9,2142.47,1078.00,1922.518709,84.1,-2.402702e+08
...,...,...,...,...,...,...,...,...,...,...,...
115,202008월,-5.824726e+07,3100.4,2.19,0.50,0.7,2326.17,1187.94,1221.939746,113.1,3.239718e+07
116,202009월,-6.441351e+07,3115.2,2.24,0.50,1.0,2327.89,1164.65,1208.597571,114.0,4.511543e+07
117,202010월,-3.751069e+07,3152.8,2.24,0.50,0.1,2267.15,1136.49,1001.099665,115.6,4.169356e+07
118,202011월,-3.147741e+07,3183.5,2.25,0.50,0.6,2591.34,1109.32,919.546249,118.3,4.231788e+07


# 전처리

In [12]:
df['next_spi'] = df['spi'].shift(-1)
df['next_spi'][-1] = 124.9

In [122]:
#전처리에 사용되었던 더미 변수들 drop
df.drop(['spi'], axis = 1, inplace = True)

In [123]:
X = df.copy()
y = X.pop('next_spi')

In [124]:
X

,sent_score,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,효준감성점수
Date,,,,,,,,
201101월,-6.268386e+07,1676.4,4.52,2.75,3.4,2069.73,1119.25,1856.746125
201102월,-6.255832e+07,1674.4,4.72,2.75,3.9,1939.30,1124.65,2046.702906
201103월,-6.957441e+07,1677.5,4.54,3.00,4.1,2106.70,1095.50,1715.795289
201104월,-7.305733e+07,1684.8,4.54,3.00,3.8,2192.36,1068.00,1635.262791
201105월,-9.356633e+07,1690.5,4.44,3.00,3.9,2142.47,1078.00,1922.518709
...,...,...,...,...,...,...,...,...
202008월,-5.824726e+07,3100.4,2.19,0.50,0.7,2326.17,1187.94,1221.939746
202009월,-6.441351e+07,3115.2,2.24,0.50,1.0,2327.89,1164.65,1208.597571
202010월,-3.751069e+07,3152.8,2.24,0.50,0.1,2267.15,1136.49,1001.099665


In [125]:
y

Date
201101월     82.5
201102월     83.4
201103월     83.9
201104월     84.1
201105월     84.4
           ...  
202008월    114.0
202009월    115.6
202010월    118.3
202011월    121.4
202012월    124.9
Name: next_spi, Length: 120, dtype: float64

In [126]:
from sklearn.model_selection import train_test_split

In [127]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [128]:
# 거시경제변수만 포함된 X 만들기
X_train_basic = X_train.drop(['sent_score','효준감성점수'], axis = 1)
X_test_basic = X_test.drop(['sent_score','효준감성점수'], axis = 1)

In [129]:
X_train_basic

,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close
Date,,,,,,
201504월,2148.1,1.99,1.75,0.4,2127.17,1076.34
202006월,3077.3,2.22,0.50,0.0,2108.33,1199.28
201912월,2912.0,1.95,1.25,0.7,2197.67,1154.07
201602월,2285.3,1.95,1.50,1.1,1916.66,1237.24
201104월,1684.8,4.54,3.00,3.8,2192.36,1068.00
...,...,...,...,...,...,...
201608월,2377.3,1.65,1.25,0.5,2034.65,1115.29
201605월,2312.8,1.91,1.50,0.8,1983.40,1188.39
202010월,3152.8,2.24,0.50,0.1,2267.15,1136.49


# 시언 감성 점수 Case

## 시언 감성 점수 제외한 거시경제변수만 포함한 성능
=> 효준 감성 점수 제외한 거시경제변수만 포함한 성능이랑 동일함

**kernel 별 RMSE 값** ( 소숫점 다섯째 자리부터 버림 )
- linear : 2.6694
- poly : 4.9715
- rbf : 5.8788
- sigmoid : 3.6421

## 시언 감성 점수 포함 성능

In [139]:
X_train_si = X_train.drop('효준감성점수',axis = 1)
X_test_si = X_test.drop('효준감성점수',axis = 1)

In [140]:
X_train_si

,sent_score,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close
Date,,,,,,,
201504월,-7.749431e+07,2148.1,1.99,1.75,0.4,2127.17,1076.34
202006월,-5.668748e+07,3077.3,2.22,0.50,0.0,2108.33,1199.28
201912월,-6.800406e+07,2912.0,1.95,1.25,0.7,2197.67,1154.07
201602월,-6.456685e+07,2285.3,1.95,1.50,1.1,1916.66,1237.24
201104월,-7.305733e+07,1684.8,4.54,3.00,3.8,2192.36,1068.00
...,...,...,...,...,...,...,...
201608월,-5.885133e+07,2377.3,1.65,1.25,0.5,2034.65,1115.29
201605월,-5.930643e+07,2312.8,1.91,1.50,0.8,1983.40,1188.39
202010월,-3.751069e+07,3152.8,2.24,0.50,0.1,2267.15,1136.49


In [141]:
X_test_si

,sent_score,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close
Date,,,,,,,
201501월,-7.296448e+07,2092.2,2.36,2.00,1.0,1949.26,1101.40
201811월,-8.226837e+07,2693.0,2.40,1.75,2.0,2096.86,1119.44
201812월,-9.637543e+07,2700.0,2.30,1.75,1.3,2041.04,1113.30
201109월,-1.038473e+08,1729.5,4.24,3.25,3.8,1769.65,1180.90
201902월,-1.019637e+08,2730.0,2.25,1.75,0.5,2195.44,1124.60
201211월,-1.538404e+08,1830.3,3.32,2.75,1.6,1932.90,1081.50
201108월,-9.440600e+07,1719.4,4.29,3.25,4.7,1880.11,1064.90
201111월,-7.628086e+07,1753.3,4.24,3.25,4.2,1847.51,1140.05
201410월,-7.599657e+07,2051.1,2.60,2.00,1.1,1964.43,1072.60


In [148]:
sc = StandardScaler()
X_train_si = sc.fit_transform(X_train_si)
X_test_si = sc.transform(X_test_si)

In [167]:
# default parameter 
svr = SVR(kernel = 'rbf')

In [168]:
svr.fit(X_train_si, y_train)

SVR()

In [169]:
y_pred = svr.predict(X_test_si)

In [170]:
RMSE = mean_squared_error(y_test, y_pred)**0.5

In [171]:
RMSE

5.953910008241991

In [172]:
del svr, RMSE

**kernel 별 RMSE 값** ( 소숫점 다섯째 자리부터 버림 )
- 거시경제변수만 사용 ==> 시언이 감성 점수 포함
- linear : 2.6694 ==> 2.6446
- poly : 4.9715 ==> 4.4179
- rbf : 5.8788 ==> 5.9539
- sigmoid : 3.6421 ==> 3.5352

# 효준 감성 점수 Case

## 효준 감성 점수 제외한 거시경제변수만 포함한 성능

In [173]:
sc = StandardScaler()
X_train_basic = sc.fit_transform(X_train_basic)
X_test_basic = sc.transform(X_test_basic)

In [174]:
# default parameter 
svr = SVR(kernel = 'sigmoid')

In [175]:
svr.fit(X_train_basic, y_train)

SVR(kernel='sigmoid')

In [176]:
y_pred = svr.predict(X_test_basic)

In [178]:
RMSE = mean_squared_error(y_test, y_pred)**0.5

In [ ]:
RMSE

In [180]:
del svr, RMSE

**kernel 별 RMSE 값** ( 소숫점 다섯째 자리부터 버림 )
- linear : 2.6694
- poly : 4.9715
- rbf : 5.8788
- sigmoid : 3.6421

## 효준 감성 점수 포함 성능

In [181]:
X_train_hj = X_train.drop('sent_score',axis = 1)
X_test_hj = X_test.drop('sent_score',axis = 1)

In [182]:
X_train_hj

,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,효준감성점수
Date,,,,,,,
201504월,2148.1,1.99,1.75,0.4,2127.17,1076.34,2212.598416
202006월,3077.3,2.22,0.50,0.0,2108.33,1199.28,1027.105971
201912월,2912.0,1.95,1.25,0.7,2197.67,1154.07,873.706855
201602월,2285.3,1.95,1.50,1.1,1916.66,1237.24,1422.961939
201104월,1684.8,4.54,3.00,3.8,2192.36,1068.00,1635.262791
...,...,...,...,...,...,...,...
201608월,2377.3,1.65,1.25,0.5,2034.65,1115.29,1454.840123
201605월,2312.8,1.91,1.50,0.8,1983.40,1188.39,1619.125596
202010월,3152.8,2.24,0.50,0.1,2267.15,1136.49,1001.099665


In [183]:
X_test_hj

,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,효준감성점수
Date,,,,,,,
201501월,2092.2,2.36,2.00,1.0,1949.26,1101.40,1850.441197
201811월,2693.0,2.40,1.75,2.0,2096.86,1119.44,1693.663795
201812월,2700.0,2.30,1.75,1.3,2041.04,1113.30,1645.223765
201109월,1729.5,4.24,3.25,3.8,1769.65,1180.90,1915.112598
201902월,2730.0,2.25,1.75,0.5,2195.44,1124.60,1629.146004
201211월,1830.3,3.32,2.75,1.6,1932.90,1081.50,2333.739232
201108월,1719.4,4.29,3.25,4.7,1880.11,1064.90,2334.489846
201111월,1753.3,4.24,3.25,4.2,1847.51,1140.05,1103.045565
201410월,2051.1,2.60,2.00,1.1,1964.43,1072.60,1950.946648


In [184]:
sc = StandardScaler()
X_train_hj = sc.fit_transform(X_train_hj)
X_test_hj = sc.transform(X_test_hj)

In [185]:
# default parameter 
svr = SVR(kernel = 'sigmoid')

In [186]:
svr.fit(X_train_hj, y_train)

SVR(kernel='sigmoid')

In [187]:
y_pred = svr.predict(X_test_hj)

In [188]:
RMSE = mean_squared_error(y_test, y_pred)**0.5

In [189]:
RMSE

3.64668654835819

In [190]:
del svr, RMSE

**kernel 별 RMSE 값** ( 소숫점 다섯째 자리부터 버림 )
- linear : 2.6694 ==> 2.5692
- poly : 4.9715 ==> 3.9581
- rbf : 5.8788 ==> 5.9429
- sigmoid : 3.6421 ==> 3.6466